In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
ratings = pd.read_csv("Ratings.csv")
books = pd.read_csv("Books.csv")
users = pd.read_csv("Users.csv")

<ipython-input-2-8b6f8e642f7d>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("/content/drive/MyDrive/Library Recommendation/Books.csv")


In [3]:
n_ratings = len(ratings)
n_books = len(ratings['ISBN'].unique())
n_users = len(ratings['User-ID'].unique())

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique ISBN: {n_books}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per book: {round(n_ratings/n_books, 2)}")

Number of ratings: 1149780
Number of unique ISBN: 340556
Number of unique users: 105283
Average ratings per user: 10.92
Average ratings per book: 3.38


In [4]:
user_freq = ratings[['User-ID', 'ISBN']].groupby(
    'User-ID').count().reset_index()
user_freq.columns = ['User-ID', 'n_ratings']
print(user_freq.head())

   User-ID  n_ratings
0        2          1
1        7          1
2        8         18
3        9          3
4       10          2


In [5]:
mean_rating = ratings.groupby('ISBN')[['Book-Rating']].mean()
# Lowest rated books
lowest_rated = mean_rating['Book-Rating'].idxmin()
books.loc[books['ISBN'] == lowest_rated]
# Highest rated books
highest_rated = mean_rating['Book-Rating'].idxmax()
books.loc[books['ISBN'] == highest_rated]
# Number of people who rated movies rated movie highest
ratings[ratings['ISBN']==highest_rated]
# Number of people who rated movies rated movie lowest
ratings[ratings['ISBN']==lowest_rated]

# Use bayesian average
books_stats = ratings.groupby('ISBN')[['Book-Rating']].agg(['count', 'mean'])
books_stats.columns = books_stats.columns.droplevel()

In [8]:
def bayesian_average(ratings_count, rating_mean):
  prior_count = 10
  prior_mean = 3
  return (ratings_count * rating_mean + prior_count * prior_mean) / (ratings_count + prior_count)

# Column for Bayesian Average Rating
books_stats['bayesian_avg'] = books_stats.apply(lambda x: bayesian_average(x['count'], x['mean']), axis=1)
top_n_books = books_stats.sort_values(by='bayesian_avg', ascending=False).head(11)
top_n_books_details = top_n_books.merge(books, on='ISBN')

print(f"\nTop {10} Books with Highest Rating:")
print(top_n_books_details[['Book-Title', 'Book-Author', 'bayesian_avg']])


Top 10 Books with Highest Rating:
                                          Book-Title            Book-Author  \
0                                               Free           Paul Vincent   
1   Harry Potter and the Chamber of Secrets (Book 2)          J. K. Rowling   
2       Harry Potter and the Goblet of Fire (Book 4)          J. K. Rowling   
3  Harry Potter and the Prisoner of Azkaban (Book 3)          J. K. Rowling   
4  The Blue Day Book: A Lesson in Cheering Yourse...  Bradley Trevor Greive   
5     Harry Potter and the Sorcerer's Stone (Book 1)          J. K. Rowling   
6       Where the Sidewalk Ends : Poems and Drawings       Shel Silverstein   
7                                          El Hobbit       J. R. R. Tolkien   
8                                         Falling Up       Shel Silverstein   
9  The Baby Book: Everything You Need to Know Abo...           Martha Sears   

   bayesian_avg  
0      7.187500  
1      6.411111  
2      6.367647  
3      6.299517  
4    